# Zero-shot


### Setting up our LLM

For this walkthrough we'll be using Falcon-7B-instruct as our base LLM. This is to keep things open-source and accessible. As you'll see, we'll also be using the free GPU environment from GCP.

Let's start by grabbing some dependencies.

In [ ]:
!pip install -q -U transformers einops accelerate peft xformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

Now we can load our model using Hugging Face's `transformers` library.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    model_kwargs={"temperature" : 0, "max_length" : 128}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

We we can use LangChain's `HuggingFacePipeline` to load our local pipeline!

In [ ]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipeline)

### Prompt Templates

First things first, let's look at a simple prompt template.

In [ ]:
from langchain import PromptTemplate

template = """
You are an expert in fashion design. Write a description for a fashion outfit in {style} style appropriate for a {role} at a {occasion}. Be sure to address the colors and the textures.
"""

prompt = PromptTemplate.from_template(template)

You can think of this prompt template in the following sense:

It is going to take some input (here we're using `style`, 'role', 'occasion'} and it's going to format the template string to include that input.


In [ ]:
prompt.format(style="bohemian", role="a petite apple shaped woman", occasion="wedding")

'\nYou are an expert in fashion design. Write a description for a fashion outfit in bohemian style appropriate for a a petite apple shaped woman at a wedding. Be sure to address the colors and the textures.\n'

Perfect!

That's exactly what we expected to see!

Now let's check out an example with 2 inputs to see how we can start extending this idea to more complex prompts.

In [ ]:
multi_input_template = """
You are an expert in fashion design. Write a description for a fashion outfit in {style} style appropriate for a {role} at a {occasion}. Be sure to address the colors and the textures.
"""

multi_input_prompt = PromptTemplate(input_variables=["style", "role", "occasion"], template=multi_input_template)

In [ ]:
multi_input_prompt.format(style="bohemian", role="a petite apple shaped woman", occasion="wedding")

'\nYou are an expert in fashion design. Write a description for a fashion outfit in bohemian style appropriate for a a petite apple shaped woman at a wedding. Be sure to address the colors and the textures.\n'

### Simple Chain

Now that we have our LLM and a prompt template, let's set up a simple chain that leverages Falcon-7B-instruct!

In [ ]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=multi_input_prompt, llm=llm)
style="bohemian"
role="a petite apple shaped woman"
occasion="wedding"

print(llm_chain.run({"style" : style, "role" : role, "occasion": occasion}))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.war


You are an expert in fashion design. Write a description for a fashion outfit in bohemian style appropriate for a a petite apple shaped woman at a wedding. Be sure to address the colors and the textures.
For a petite apple-shaped woman attending a bohemian-style wedding, a suitable outfit could consist of a floaty, floral-print maxi dress paired with a delicate, cream-colored shawl. The dress should be accessorized with earthy-hued, boho-style jewelry, such as a pendant necklace, beaded bracelets, and hoop earrings. The shawl can be a neutral, earthy-hued


Spot on!

Let's try another example!

In [ ]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=multi_input_prompt, llm=llm)
style="gothic"
role="a man"
occasion="business meeting"

print(llm_chain.run({"style" : style, "role" : role, "occasion": occasion}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



You are an expert in fashion design. Write a description for a fashion outfit in gothic style appropriate for a a man at a business meeting. Be sure to address the colors and the textures.
The man's outfit in gothic style for a business meeting comprises a tailored black or dark blue blazer, a crisp white dress shirt, a black or dark red tie, and black or brown dress shoes. The blazer should be adorned with intricate silver or gold embellishments, such as buttons or lapels. The dress shirt should be made of fine quality silk or cotton, and should be tucked in to the tailored blazer. The tie


Let's provide our input from our csv.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install pandas

First, we will generate the results for people by just providing the gender.

In [ ]:
import pandas as pd

# Read CSV file
simple_types = "simple_types.csv"  # Provide the path to your CSV file
simple_types_df = pd.read_csv(simple_types)
simple_types_df

,Style,Type,Event
0,bohemian,man,a casual date
1,bohemian,woman,a halloween party
2,classic,man,a bar
3,classic,woman,a work / office event
4,gothic,man,a work / office event
...,...,...,...
95,gothic,woman,a business lunch
96,sporty,man,a conference
97,sporty,woman,a beach
98,casual,man,a halloween party


In [ ]:
# Initialize a list to store results
results = []

for index, row in simple_types_df.iterrows():
    style = row['Style']
    role = row['Type']
    occasion = row['Event']
    # Generate outfit description
    output = llm_chain.run({"style": style, "role": role, "occasion": occasion})
    results.append({'style': style, 'role': role, 'occasion': occasion, 'description': output})



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` 

In [ ]:
# Convert list of results to DataFrame
results_df = pd.DataFrame(results)
results_df

,style,role,occasion,description
0,bohemian,man,a casual date,\nYou are an expert in fashion design. Write a...
1,bohemian,woman,a halloween party,\nYou are an expert in fashion design. Write a...
2,classic,man,a bar,\nYou are an expert in fashion design. Write a...
3,classic,woman,a work / office event,\nYou are an expert in fashion design. Write a...
4,gothic,man,a work / office event,\nYou are an expert in fashion design. Write a...
...,...,...,...,...
95,gothic,woman,a business lunch,\nYou are an expert in fashion design. Write a...
96,sporty,man,a conference,\nYou are an expert in fashion design. Write a...
97,sporty,woman,a beach,\nYou are an expert in fashion design. Write a...
98,casual,man,a halloween party,\nYou are an expert in fashion design. Write a...


In [ ]:
# Assuming new_df is your DataFrame

# Extract the desired part of the text from the 'context' column
results_df['final_description'] = results_df['description'].str.split("Be sure to address the colors and the textures.").str[1]

# Print the DataFrame with the extracted context
results_df

,style,role,occasion,description,final_description
0,bohemian,man,a casual date,\nYou are an expert in fashion design. Write a...,\nThe bohemian fashion outfit for a casual dat...
1,bohemian,woman,a halloween party,\nYou are an expert in fashion design. Write a...,\nThe bohemian woman's outfit for a Halloween ...
2,classic,man,a bar,\nYou are an expert in fashion design. Write a...,\nThe gentleman dons a classic black tailored ...
3,classic,woman,a work / office event,\nYou are an expert in fashion design. Write a...,\nA classic and sophisticated work outfit for ...
4,gothic,man,a work / office event,\nYou are an expert in fashion design. Write a...,\nThe man's gothic outfit consists of a tailor...
...,...,...,...,...,...
95,gothic,woman,a business lunch,\nYou are an expert in fashion design. Write a...,\nThe woman's gothic business lunch outfit con...
96,sporty,man,a conference,\nYou are an expert in fashion design. Write a...,\nThe fashion outfit consists of a tailored bl...
97,sporty,woman,a beach,\nYou are an expert in fashion design. Write a...,"\nThe woman is wearing a white, sleeveless, sc..."
98,casual,man,a halloween party,\nYou are an expert in fashion design. Write a...,"\nFor a casual Halloween outfit, consider a bl..."


In [ ]:
# Specify the output CSV file name
output_csv_file = "zero-shot_simple-types.csv"

# Save DataFrame to CSV file
results_df.to_csv(output_csv_file, index=False)

Now, let's give more complex types by including information about the bodytype.

In [ ]:
# Read CSV file
complex_types = "complex_types.csv"  # Provide the path to your CSV file
complex_types_df = pd.read_csv(complex_types)
complex_types_df

,Style,Type,Event
0,bohemian,"a small-framed, delicate woman",a work / office event
1,bohemian,"a short, curvy man with a muscular build",a casual date
2,classic,"a small-framed, delicate woman",a play / concert
3,classic,"a short, curvy man with a muscular build",a work / office event
4,gothic,"a small-framed, delicate woman",a beach
...,...,...,...
95,gothic,"a short, curvy man with a muscular build",a birthday party
96,sporty,"a small-framed, delicate woman",a wedding
97,sporty,"a short, curvy man with a muscular build",a night club
98,casual,"a small-framed, delicate woman",a business meeting


In [ ]:
# Initialize a list to store results
complex_results = []

for index, row in complex_types_df.iterrows():
    style = row['Style']
    role = row['Type']
    occasion = row['Event']
    # Generate outfit description
    output = llm_chain.run({"style": style, "role": role, "occasion": occasion})
    complex_results.append({'style': style, 'role': role, 'occasion': occasion, 'description': output})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` 

In [ ]:
# Convert list of results to DataFrame
complex_results_df = pd.DataFrame(complex_results)
complex_results_df

,style,role,occasion,description
0,bohemian,"a small-framed, delicate woman",a work / office event,\nYou are an expert in fashion design. Write a...
1,bohemian,"a short, curvy man with a muscular build",a casual date,\nYou are an expert in fashion design. Write a...
2,classic,"a small-framed, delicate woman",a play / concert,\nYou are an expert in fashion design. Write a...
3,classic,"a short, curvy man with a muscular build",a work / office event,\nYou are an expert in fashion design. Write a...
4,gothic,"a small-framed, delicate woman",a beach,\nYou are an expert in fashion design. Write a...
...,...,...,...,...
95,gothic,"a short, curvy man with a muscular build",a birthday party,\nYou are an expert in fashion design. Write a...
96,sporty,"a small-framed, delicate woman",a wedding,\nYou are an expert in fashion design. Write a...
97,sporty,"a short, curvy man with a muscular build",a night club,\nYou are an expert in fashion design. Write a...
98,casual,"a small-framed, delicate woman",a business meeting,\nYou are an expert in fashion design. Write a...


In [ ]:
# Assuming new_df is your DataFrame

# Extract the desired part of the text from the 'context' column
complex_results_df['final_description'] = complex_results_df['description'].str.split("Be sure to address the colors and the textures.").str[1]

# Print the DataFrame with the extracted context
complex_results_df

,style,role,occasion,description,final_description
0,bohemian,"a small-framed, delicate woman",a work / office event,\nYou are an expert in fashion design. Write a...,"\nA delicate, bohemian-style outfit for a smal..."
1,bohemian,"a short, curvy man with a muscular build",a casual date,\nYou are an expert in fashion design. Write a...,"\nFor a bohemian-chic date night, opt for a co..."
2,classic,"a small-framed, delicate woman",a play / concert,\nYou are an expert in fashion design. Write a...,"\nThe delicate woman should opt for a classic,..."
3,classic,"a short, curvy man with a muscular build",a work / office event,\nYou are an expert in fashion design. Write a...,"\nA classic, tailored fit is essential for a s..."
4,gothic,"a small-framed, delicate woman",a beach,\nYou are an expert in fashion design. Write a...,"\nThe delicate, small-framed woman should opt ..."
...,...,...,...,...,...
95,gothic,"a short, curvy man with a muscular build",a birthday party,\nYou are an expert in fashion design. Write a...,"\nFor a special occasion like a birthday, a go..."
96,sporty,"a small-framed, delicate woman",a wedding,\nYou are an expert in fashion design. Write a...,"\nFor a delicate woman attending a wedding, a ..."
97,sporty,"a short, curvy man with a muscular build",a night club,\nYou are an expert in fashion design. Write a...,"\nThe curvy man should opt for a slim-fitting,..."
98,casual,"a small-framed, delicate woman",a business meeting,\nYou are an expert in fashion design. Write a...,"\nA delicate, petite woman can pull off a styl..."


In [ ]:
# Specify the output CSV file name
output_csv_file = "zero-shot_complex-types.csv"

# Save DataFrame to CSV file
complex_results_df.to_csv(output_csv_file, index=False)